# EU climate change mitigation targets compromise forest ecosystem services and biodiversity

## Sweden

Above the code cells, there will be instructions how the users should modify the codes in the cells. If there are no instructions, then by default no changes should be needed for the cell.

## Read the data

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/py_class")

import multiFunctionalOptimizationSwe as MFO

In [ ]:
from importlib import reload
reload(MFO)

You can choose solver by defining "solver=XXX" in the argument. Possible options are "CPLEX", "CLP" and "GLOP"

In [ ]:
mfo = MFO.MultiFunctionalOptimization(solver = "CLP")

Define CC scenario data

In [ ]:
RCP = "RCP0"
objectives_globiom = 'globiom_RCP0_V2'

In [ ]:
#RCP = "RCP45"
#objectives_globiom = 'globiom_RCP45_V2'

1.5 degrees = RCP0 (no CC), NDC = RCP 4.5

In [ ]:
scenario = "MF"

In [ ]:
import wget
import os
import numpy as np
import pandas as pd

NoCC:

In [ ]:
%%time
mfo.readData("Sweden_5%_RCP0.zip",
             standsEnu = "Description", regimesEnu = ["ControlCategoryName", "AlternativeNo"], timeEnu = "period",
             sampleRatio = 1, #If no sample ratio given, the ratio is assumed to be 1
             areaCol = "RepresentedArea"
            )

RCP45:

In [ ]:
#%%time
#mfo.readData("Sweden_5%_RCP45.csv",
#             standsEnu = "Description", regimesEnu = ["ControlCategoryName", "AlternativeNo"], timeEnu = "period",
#             sampleRatio = 1,
#             areaCol = "RepresentedArea"
#            )

Remove forestry regimes connected to intensification:

In [ ]:
indexNames = mfo.data[(mfo.data['ControlCategoryName'] == 'Int_Prod') | (mfo.data['ControlCategoryName'] == 'Int_HybridExotic') | (mfo.data['ControlCategoryName'] == 'Int_Contorta')].index
mfo.data.drop(indexNames, inplace = True)

Create a column for Pulpfuel i.e. pulp plus firewood

In [ ]:
mfo.data['PulpFuel'] = mfo.data.SumPulpVolumeTotal.values + mfo.data.SumHarvestFuelwoodTotal.values

Create column for simulated harvest for globiom optimisation

In [ ]:
mfo.data['SimulatedSAWlog'] = mfo.data.SumTimberVolumeTotal.values/5

In [ ]:
mfo.data['SimulatedResidue'] = mfo.data.SumHarvestResiduesTotal.values/5

In [ ]:
mfo.data['SimulatedPulPFuel'] = mfo.data.PulpFuel.values/5

Create a new column for old decidious forest: older than 80 years and more than 30% deciduous

In [ ]:
mfo.data['DeciduousRatio'] = mfo.data.VolumeDecidous.values/mfo.data.StandingVolume.values

In [ ]:
mfo.data["old_deciduous_rich_forest_area"] = (mfo.data['DeciduousRatio'].values>0.3)*(mfo.data["Age"].values>80)*mfo.data["RepresentedArea"].values

Create bolean indicator for set asides to calculate share of forest set aside from management,
for managed forest to calculate values only for managed forest, and for CCF to calculate share of CCF

In [ ]:
mfo.data['SetAside'] = np.where(mfo.data.ControlCategoryName == 'SetAside (Unmanaged)', True, False)

In [ ]:
mfo.data['managed'] = np.where(mfo.data.ControlCategoryName == 'SetAside (Unmanaged)', False, True)

In [ ]:
mfo.data['CCF'] = np.where(mfo.data.ControlCategoryName == 'CCF', True, False)

Manual calculation of Total values:

In [ ]:
mfo.data["Total_VolumeDeciduous"] = mfo.data["VolumeDecidous"] * mfo.data["RepresentedArea"]
mfo.data["Total_DeadWoodVolume"] = mfo.data["DeadWoodVolume"] * mfo.data["RepresentedArea"]
mfo.data["Total_RecreationIndex"] = mfo.data["RecreationIndex"] * mfo.data["RepresentedArea"]
mfo.data["Total_TotalCarbon"] = mfo.data["TotalCarbon"] * mfo.data["RepresentedArea"]
mfo.data["Total_SimulatedSAWlog"] = mfo.data["SimulatedSAWlog"] * mfo.data["RepresentedArea"]
mfo.data["Total_SimulatedPulPFuel"] = mfo.data["SimulatedPulPFuel"] * mfo.data["RepresentedArea"]
mfo.data["Total_SimulatedResidue"] = mfo.data["SimulatedResidue"] * mfo.data["RepresentedArea"]

Set indicator values for BD indicators to 0 on set asides to only calculate with values on managed

In [ ]:
mfo.data["Total_VolumeDeciduous_managed"] = mfo.data["Total_VolumeDeciduous"] * mfo.data["managed"]
mfo.data["Total_DeadWoodVolume_managed"] = mfo.data["Total_DeadWoodVolume"] * mfo.data["managed"]
mfo.data["old_deciduous_rich_forest_area_managed"] = mfo.data["old_deciduous_rich_forest_area"] * mfo.data["managed"]

## Finalise data:

In [ ]:
mfo.finalizeData(initialTime=0, initialRegime="ControlCategoryNameInitial state_AlternativeNo1")

## Start defining the optimization problem

#### Define objectives

Objective format: Unique_key : (Long human readable name,column name in data, max/min objective, year wise aggregation, stand wise aggregation [, target year])

Options for "objective": "max"imise or "min"imise it
year wise aggregation: "min" (minimum value), "average", "firstYear", "sum", "targetYearWithSlope","targetYear"
stand wise aggregation: "sum", "areaWeightedAverage", "areaWeightedSum"
targe yeart: any year except the first one

Objective dictionary structure: "Unique short name":("Long human readable name","column name in the data")

In [ ]:
Wood = {
"NetPresentValue":("Total sum net present value of cut forest","NPV","max","firstYear","areaWeightedSum"),
"TotalAnnIncrement":("Annual Increment (maximised min over all years)","AnnualIncrementNetTotal","max","min","areaWeightedAverage"),
"HarvestEvenFlow":("Average harvest","SumVolumeCutTotal","max","min","areaWeightedAverage")
}

In [ ]:
Recreation = { 
"RecreationIndex":("No decrease in recreation index","Relative_Total_RecreationIndex","max","min","sum")
}

In [ ]:
Climate = { 
"TotalCarbon":("No decrease carbon stocks","Relative_Total_TotalCarbon","max","min","sum")
}

In [ ]:
Biodiversity = { 
"DeadWoodVolume":("60% incr in deadwood by 2050","Relative_Total_DeadWoodVolume_managed","max","targetYearWithSlope","sum",8),  
"OldDeciduous":("60% incr in old deciduous area by 2050","Relative_old_deciduous_rich_forest_area_managed","max","targetYearWithSlope","sum",8),
"SetAside":("Share of set aside forest","SetAside","max","firstYear","areaWeightedAverage")
}

In [ ]:
Resilience = {
"DeciduousVolume":("60% incr in deciduous volume by 2050","Relative_Total_VolumeDeciduous_managed","max","targetYearWithSlope","sum",8)
}

In [ ]:
Water = {
"CCF":("Share of CCF","CCF","max","firstYear","areaWeightedAverage")
}

In [ ]:
objectives = {
    **Wood,
    **Recreation,
    **Climate,
    **Biodiversity,
    **Resilience,
    **Water
}

In [ ]:
mfo.defineObjectives(objectives)

GLOBIOM demands VERSION 2 - option for assortment transfer Attention: Has to be run after defining the objectives!

In [ ]:
# 1.5 degree scenario; matches with RCP 0 (no CC)
if objectives_globiom == 'globiom_RCP0_V2':
    
    demands = pd.read_csv("G1p5_5%.csv") 

    print("used RCP0")
      
# NDC scenario, matches with RCP 4.5
elif objectives_globiom == 'globiom_RCP45_V2':
    
    demands = pd.read_csv("G4p5_5%.csv") 

    print("used RCP4.5")

In [ ]:
if objectives_globiom == 'globiom_RCP0_V2' or objectives_globiom =='globiom_RCP45_V2' :
    
    sawlog = demands["GSawlog_uB"]
    sawlog = sawlog.to_list()
    
    pulpfuel = demands["GPulpFuel_uB"]
    pulpfuel = pulpfuel.to_list()

    residues = demands["GResidues"]
    residues = residues.to_list()
    
    mfo.addGlobiomTargets(
        {
        "log": sawlog,
        "pulp": pulpfuel,
        "residues": residues
        },
        {
            #Log is converted primarily into log, and sencondary into pulp; no transferrate, both are volumes under bark
            "Total_SimulatedSAWlog":
                {"log":[1,"primary"],"pulp":[1,"secondary"], "residues":[1.136,"secondary"]},
                                          
            # Pulp is converted primarily into pulp, and sencondary into residues; 
            # with transferrate 1.136, because pulp volume is under bark, residues are over bark (barkfactor = 1.136)  
            "Total_SimulatedPulPFuel":
                {"pulp":[1,"primary"], "residues":[1.136,"secondary"]}, 
            
            "Total_SimulatedResidue":
                {"residues":[1,"primary"]} #Biomass only to residues
        },
        
        # ----------------
        # by default exactMatching is FALSE
        # ----------------
        # Functionality was implemented for Norway and their nationl policy scenarios
        # NOT required for Cross-scale analysis V2
        exactMatching=False   
        
    )
    
    print("objective function with assortment transfer loaded")

## Calculate objective ranges

In [ ]:
%%time
mfo.calculateObjectiveRanges()

In [ ]:
mfo.objectiveRanges

## Export the objetive ranges

Can save re-calculation times if big data sets are optimised

In [ ]:
import json
mfo.objectiveRanges

with open("objectiveRanges_V2_"+RCP+"_"+scenario+".json", "w") as json_file:
    json.dump(mfo.objectiveRanges, json_file)

Save the objectives ranges also as CSV

In [ ]:
import pandas
df = pandas.read_json("objectiveRanges_V2_"+RCP+"_"+scenario+".json")
df.to_csv("objectiveRanges_V2_"+RCP+"_"+scenario+".csv")

## Show the GUI

In [ ]:
mfo.showGUI(debug=True)

## Export solution data as csv

In [ ]:
import os

b = []
c = []
for key in mfo.regimesDecision.keys():
    if mfo.regimesDecision[key].solution_value() > 0:
        b = b+ [(key[0],x, key[1]) for x in range(0,21)]
        c = c+ [(key[0],key[1],mfo.regimesDecision[key].solution_value())]
data2b = mfo.data.iloc[mfo.data.index.isin(b)]
data2b.to_csv("solution_alldata_V2_"+RCP+"_"+scenario+"_data.csv")
c1 = pd.DataFrame(c)
c1.to_csv("solution_V2_"+RCP+"_"+scenario+"_solutions.csv")

## Export objective values

The optimal solution for each objective.

In [ ]:
with open("objectiveValues_V2_"+scenario+'_'+RCP+".csv","w") as file: 
    delim = "" 
    for objName in mfo.objectiveTypes.keys(): 
        file.write(delim+objName) 
        delim = "," 
    file.write("\n") 
    delim = "" 
    for objName in mfo.objectiveTypes.keys(): 
        file.write(delim+str(mfo.objective[objName].solution_value())) 
        delim = "," 
    file.write("\n")